In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from os.path import basename
from blab.array import loadmat
from glob import glob
import mne
import bct
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.sparse import csc_matrix
import scipy.io as spio
from tqdm import tqdm
from statsmodels.stats.multitest import multipletests
import networkx as nx
import scipy.sparse as sparse
from IPython.display import display as disp
from scipy import special
from scipy import ndimage
from mne.stats.cluster_level import _find_clusters as find_clusters
from toolz.functoolz import curry
from scipy.stats._hypotests import _batch_generator as batch_generator
from pickle import dump, load

In [39]:
mbrt_df = pd.read_csv('mbrt_all.csv')
mbrt_df.set_index(['Day', 'Session', 'Subject'], inplace=True)
mbrt_im1 = mbrt_df.query('Subject != 10 and Session == "Im1"')['MBRT ANY'].to_list()
mbrt_im1 = np.array(mbrt_im1[15:] + mbrt_im1[:15])

mbrt_im2 = mbrt_df.query('Subject != 10 and Session == "Im2" and Day == "TMS"')['MBRT ANY'].to_list()[:14]
mbrt_im2 = np.array(mbrt_im2)

fnames_im1 = sorted(glob('../data/fnc/fnc_im1/*'))
fnames_im2 = sorted(glob('../data/fnc/fnc_im2/*'))

In [40]:
def corr(src, trgt):
    res = []
    mask = ~np.isnan(trgt)
    src = src.copy()[..., mask]
    trgt = trgt.copy()[mask]
    for i in src.reshape(116*116, -1):
        if np.all(i == i[0]):
            res.append((np.nan, np.nan))
        else:
            # temp = stats.permutation_test([i, trgt], lambda *o: stats.pearsonr(*o)[0], permutation_type='pairings', n_resamples=1000)
            stat, pval = stats.pearsonr(i, trgt)
            # stat, pval = temp.statistic, temp.pvalue
            res.append((stat, pval))
    return np.array(res).reshape(116, 116, 2)

res_matr = {}
for fname in tqdm(fnames_im1):
    sas = loadmat(fname)
    sos = sas['nbs']
    cons = sos['conmatr']
    n = cons.shape[-1]//2
    cons = cons[..., :n] - cons[..., n:] # post - base
    corr_mat = corr(cons, mbrt_im1)
    res_matr[basename(fname)] = corr_mat
    datar = corr_mat[..., 0].copy()
    a = np.tril_indices(116)
    corr_mat[a] *= np.nan
    # corr_mat[..., 1] = multipletests(corr_mat[..., 1], alpha=0.05, method='hs', is_sorted=False, returnsorted=False)[-1]
    inds = np.arange(116)
    a, b = np.meshgrid(inds, inds)
    mask = (corr_mat[..., 1] < 0.05)
    row, col = a[mask], b[mask]
    datap = corr_mat[..., 1][mask]
    inds = np.argsort(datap)
    cut = len(inds)
    datap = datap[inds][:cut]
    row, col = row[inds][:cut], col[inds][:cut]

    # datar = csc_matrix((datar, (row, col)), shape=(116, 116))
    datap = csc_matrix((np.ones(cut), (row, col)), shape=(116, 116))
    sas['nbs']['NBS']['con_mat'] = datap
    sas['nbs']['NBS']['test_stat'] = datar
    # spio.savemat(f'{fname}_UPD.mat', sas)

100%|██████████| 5/5 [00:04<00:00,  1.03it/s]


In [175]:
inds = np.arange(116)
a, b = np.meshgrid(inds, inds)
mask = (corr_mat[..., 1] < 0.05)
inds = np.vstack([a[mask], b[mask]])
arr = np.hstack([inds.T, corr_mat[..., 0][mask][..., None], corr_mat[..., 1][mask][..., None]])
df = pd.DataFrame(arr, columns=['Node.A', 'Node.B', 'Corr. coef.', 'p-value'])
df.sort_values('p-value')

,Node.A,Node.B,Corr. coef.,p-value
38,56.0,13.0,-0.558739,0.000726
186,102.0,87.0,-0.517674,0.002032
107,102.0,39.0,-0.516568,0.002086
137,60.0,53.0,0.515664,0.002131
67,87.0,19.0,-0.514930,0.002167
...,...,...,...,...
86,74.0,33.0,-0.345784,0.048717
62,52.0,19.0,0.344991,0.049270
199,105.0,99.0,-0.344818,0.049392
12,113.0,3.0,0.344786,0.049415


In [283]:
for fname in tqdm(fnames_im1):
    sas = loadmat(fname)
    sos = sas['nbs']
    cons = sos['conmatr']
    n = cons.shape[-1]//2
    cons = cons[..., :n] - cons[..., n:] # post - base
    corr_mat = res_matr[basename(fname)].copy()
    a = np.tril_indices(116)
    corr_mat[a] = np.swapaxes(corr_mat, 0, 1)[a]
    datar = corr_mat[..., 0].copy()
    a = np.tril_indices(116)
    corr_mat[a] *= np.nan
    inds = np.arange(116)
    a, b = np.meshgrid(inds, inds)
    mask = (corr_mat[..., 1] < 0.025)
    mask_neg = mask & (corr_mat[..., 0] < 0)
    mask_pos = mask & (corr_mat[..., 0] > 0)
    rown, rowp, coln, colp = a[mask_neg], a[mask_pos], b[mask_neg], b[mask_pos]
    datapn, datapp = corr_mat[..., 1][mask_neg], corr_mat[..., 1][mask_pos]
    indsn, indsp = np.argsort(datapn), np.argsort(datapp)
    datapn, datapp = datapn[indsn], datapp[indsp]
    rown, rowp, coln, colp = rown[indsn], rowp[indsp], coln[indsn], colp[indsp]

    datapn = csc_matrix((np.ones(len(indsn)), (rown, coln)), shape=(116, 116))
    datapp = csc_matrix((np.ones(len(indsp)), (rowp, colp)), shape=(116, 116))

    sas['nbs']['NBS']['con_mat'] = datapn
    sas['nbs']['NBS']['test_stat'] = datar
    # spio.savemat(f'{fname}_UPD_NEG.mat', sas)
    sas['nbs']['NBS']['con_mat'] = datapp
    # spio.savemat(f'{fname}_UPD_POS.mat', sas)

100%|██████████| 5/5 [00:01<00:00,  3.31it/s]


In [488]:
def edge_adjacency(n=116):
    inds = np.arange(n, dtype=int)
    nan_inds = np.tril_indices(n)
    a, b = np.meshgrid(inds, inds)
    edge_pairs = np.stack([a[nan_inds], b[nan_inds]])
    edge_pairs = edge_pairs.astype(np.uint16).T
    edge_inds = np.arange(edge_pairs.shape[0], dtype=np.uint16)
    a, b = np.meshgrid(edge_inds, edge_inds)
    a, b = edge_pairs[a], edge_pairs[b]
    adj = (a[..., 0] == b[..., 0]) | (a[..., 0] == b[..., 1]) | (a[..., 1] == b[..., 0]) | (a[..., 1] == b[..., 1])
    a, b = None, None
    adj = sparse.coo_matrix(adj)
    return adj

adj = edge_adjacency(116)

In [527]:
r_pdf(np.r_[0.1, 0.2, 0.3], 10)

array([0.78342441, 0.579584  , 0.39969147])

In [5]:
def r_ppf(q, n):
    ab = n/2 - 1
    return -(special.btdtri(ab, ab, q/2)*2 - 1)

def r_pdf(r, n):
    ab = n/2 - 1
    return 2*special.btdtr(ab, ab, 0.5*(1 - abs(np.float64(r))))

def pearsonr_no_p(x, y, axis):
    assert len(x.shape) == len(y.shape)
    x_m = x.mean(axis, keepdims=True)
    y_m = y.mean(axis, keepdims=True)
    x_sq = np.sum(np.square(x - x_m), axis=axis)
    y_sq = np.sum(np.square(y - y_m), axis=axis)
    num = np.sum((x - x_m)*(y - y_m), axis=axis)
    denum = np.sqrt(x_sq*y_sq)
    res = num/denum
    res[np.isnan(res)] = 0
    res = np.maximum(np.minimum(res, 1), -1)
    return res

def clean_clusters(clusters, sums, fun=lambda x, y: True):
    new_clusters = []
    new_sums = []
    for ind, cl in enumerate(clusters):
        if len(cl) >= 2 and fun(cl, sums[ind]):
            new_clusters.append(cl)
            new_sums.append(sums[ind])
    return new_clusters, new_sums

def cluster_inds(clusters, perminds):
    clus_inds = []
    for cl in clusters:
        temp = perminds[cl[0]]
        clus_inds.append(temp)
    return np.array(clus_inds)

def calc_pval(arr, y):
    denum = len(arr)
    sign = np.sign(y)
    if sign < 0:
        num = np.sum(np.abs(arr[arr < 0]) <= np.abs(y))
    else:
        num = np.sum(np.abs(arr[arr > 0]) <= np.abs(y))
    return num/denum

@curry
def compare_tail(arr, crit, x, y):
    return calc_pval(arr, y) <= crit

def perm_test(x, y, threshold, adj, permutations=1000, batch_num=250, tail=0, clusteralpha=0.05):
    # x [n_samples, 1] y [n_samples, k]
    print(x.shape, y.shape)
    n_samples = x.shape[0]
    inds = np.arange(n_samples, dtype=int)
    perms = [np.random.permutation(inds) for i in range(permutations)]
    observed_stat = pearsonr_no_p(x, y, axis=0)
    null_dist = []
    for batch in batch_generator(perms, batch_num):
        perms = len(batch)
        perms_adj = sparse.coo_array((perms, perms), dtype=int)
        temp_adj = mne.stats.combine_adjacency(perms_adj, adj)

        batch = np.array(batch)
        chunk = x[batch]
        null_part = pearsonr_no_p(chunk, y[None, :], axis=1)
        perms_inds = np.arange(null_part.shape[0]).repeat(null_part.shape[1])

        clus, sums = find_clusters(null_part.flatten(), threshold=threshold, adjacency=temp_adj, tail=tail)
        clus, sums = clean_clusters(clus, sums)
        clus_inds = cluster_inds(clus, perms_inds)

        max_sums = np.empty(clus_inds.max() + 1)
        np.maximum.at(max_sums,clus_inds,sums)

        null_dist += max_sums[clus_inds].tolist()

    null_dist = np.array(null_dist).flatten()

    clus, sums = find_clusters(observed_stat, threshold=threshold, adjacency=adj, tail=tail)
    fun = compare_tail(null_dist, clusteralpha)
    clus, sums = clean_clusters(clus, sums, fun=fun)
    pvals = [calc_pval(null_dist, s) for s in sums]

    return clus, pvals

# mbrt_df = pd.read_csv('mbrt_all.csv').query('Day == "TMS" and Subject != 10')[['Subject', 'Session', 'MBRT ANY']]
# mbrt_df = mbrt_df.set_index(['Subject', 'Session']).unstack()
# mbrt_df.columns = ['Im1', 'Im2']
# mbrt = (mbrt_df['Im2'] - mbrt_df['Im1']).to_numpy()[:14]
#
# nanmask = ~np.isnan(mbrt)
# mbrt = mbrt[nanmask]
# mbrt = mbrt.reshape(-1, 1)
#
# base_str = '../data/fnc/Conn_Stat_Corr_TMS_Im2-Im1/NBS_'
# bands = ['Theta', 'Alpha', 'Alpha12', 'Alpha2', 'Beta']
# cases = ['b', 'p1', 'p2', 'rest']
#
# results = {}
# for band in bands:
#     for case in cases:
#         print(f'{band}, {case}')
#         conn = loadmat(f'{base_str}{band}_{case}.mat')['nbs']['conmatr']
#         conn = conn[..., :14] - conn[..., 14:]
#         conn = conn[..., nanmask]
#         conn_flat = conn[np.tril_indices(116)].T
#         sas = perm_test(mbrt, conn_flat, r_ppf(0.025, 12), adj, permutations=1000, batch_num=250, tail=0)
#         results[(band, case)] = sas
#         f = open('stats.pickle', 'wb')
#         dump(results, f)
#         f.close()

In [2]:
mbrt_df = pd.read_csv('mbrt_all.csv').query('Day == "TMS" and Subject != 10')[['Subject', 'Session', 'MBRT ANY']]
mbrt_df = mbrt_df.set_index(['Subject', 'Session']).unstack()
mbrt_df.columns = ['Im1', 'Im2']
mbrt = (mbrt_df['Im2'] - mbrt_df['Im1']).to_numpy()[:14]

nanmask = ~np.isnan(mbrt)
mbrt = mbrt[nanmask]
mbrt = mbrt.reshape(-1, 1)

In [34]:
f = open('stats.pickle', 'rb')
results = load(f)
f.close()

base_str = '../data/fnc/Conn_Stat_Corr_TMS_Im2-Im1/NBS_'
base_str_save = '../data/fnc/Conn_Stat_Corr_TMS_Im2-Im1/results/NBS_'
bands = ['Theta', 'Alpha', 'Alpha12', 'Alpha2', 'Beta']
cases = ['b', 'p1', 'p2', 'rest']

for band in bands:
    for case in cases:
        print(f'{band}, {case}')
        mat = loadmat(f'{base_str}{band}_{case}.mat')
        conn = mat['nbs']['conmatr']
        conn = (conn[..., :14] - conn[..., 14:])[..., nanmask]
        corr = pearsonr_no_p(mbrt, conn[np.tril_indices(116)].T, axis=0)

        res = np.zeros(shape=(116, 116), dtype=float)
        res[np.tril_indices(116)] = corr
        res = res.T
        res[np.tril_indices(116)] = corr

        clusters = results[(band, case)][0]
        cl_inds_pos = []
        cl_inds_neg = []
        for i in clusters:
            if corr[i][0] < 0:
                cl_inds_neg += i.tolist()
            else:
                cl_inds_pos += i.tolist()
        cl_inds_pos = np.array(cl_inds_pos).astype(int)
        cl_inds_neg = np.array(cl_inds_neg).astype(int)




        inds = np.arange(116)
        a, b = np.meshgrid(inds, inds)

        mask_pos = np.zeros(shape=(116, 116), dtype=bool)
        mask_neg = np.zeros(shape=(116, 116), dtype=bool)
        tril_inds = np.tril_indices(116)
        a_tril, b_tril = a[tril_inds], b[tril_inds]
        mask_pos[a_tril[cl_inds_pos], b_tril[cl_inds_pos]] = True
        mask_neg[a_tril[cl_inds_neg], b_tril[cl_inds_neg]] = True

        row_pos, col_pos = a[mask_pos], b[mask_pos]
        row_neg, col_neg = a[mask_neg], b[mask_neg]
        datap_pos = r_pdf(res[mask_pos], 12)
        datap_neg = r_pdf(res[mask_neg], 12)

        datap_pos = csc_matrix((np.ones(len(datap_pos)), (row_pos, col_pos)), shape=(116, 116))
        datap_neg = csc_matrix((np.ones(len(datap_neg)), (row_neg, col_neg)), shape=(116, 116))

        mat['nbs']['NBS']['con_mat'] = datap_pos
        mat['nbs']['NBS']['test_stat'] = res

        spio.savemat(f'{base_str_save}{band}_{case}_POS.mat', mat)

        mat['nbs']['NBS']['con_mat'] = datap_neg

        spio.savemat(f'{base_str_save}{band}_{case}_NEG.mat', mat)

Theta, b
Theta, p1
Theta, p2


/var/folders/tj/lr6p2mw95bqcmgs698v6nqx40000gn/T/ipykernel_4118/3824740155.py:17: RuntimeWarning: invalid value encountered in true_divide
  res = num/denum


Theta, rest
Alpha, b
Alpha, p1
Alpha, p2
Alpha, rest
Alpha12, b
Alpha12, p1
Alpha12, p2
Alpha12, rest
Alpha2, b
Alpha2, p1
Alpha2, p2
Alpha2, rest
Beta, b
Beta, p1
Beta, p2
Beta, rest


In [35]:
datap_neg

<116x116 sparse matrix of type '<class 'numpy.float64'>'
	with 47 stored elements in Compressed Sparse Column format>

In [17]:
import scipy.io as spio
import numpy as np
import pandas as pd
from scipy import stats
a, b = spio.loadmat('../data/fnc/Con_Frontal_Mid_L_Precuneus_R_PCC_D2_Im1_Base_0_5_4_5s_D2_Im2_Base.mat')['Con'].reshape(2, -1)
# stats.permutation_test([(b - a)[nanmask], mbrt[:, 0]], lambda x, y: pearsonr_no_p(x[:, None], y[:, None], axis=0), permutation_type='pairings')
mbrt = pd.read_csv('mbrt_all_march10_stat_cluster_delta.csv')\
    .query('Day == "D2" and Subject != 10')['MBRT ANY'].to_numpy()
mask = ~np.isnan(mbrt)
a, b, mbrt  = a[mask], b[mask], mbrt[mask]

stats.(b, mbrt)

SpearmanrResult(correlation=0.1328671328671329, pvalue=0.6805975082860818)

In [13]:
a.shape

(12,)

In [478]:
width = 40
n_subjects = 10
signal_mean = 100
signal_sd = 100
noise_sd = 0.01
gaussian_sd = 5
sigma = 1e-3  # sigma for the "hat" method
n_permutations = 'all'  # run an exact test
n_src = width * width

# For each "subject", make a smoothed noisy signal with a centered peak
rng = np.random.RandomState(2)
rngY = np.random.RandomState(3)
X = noise_sd * rng.randn(n_subjects, width, width)
Y = noise_sd * rngY.randn(n_subjects, width, width)
X[:, width // 2, width // 2] = signal_mean + rng.randn(n_subjects) * signal_sd
size = width // 2 - 1
gaussian = np.exp(-(np.arange(-size, size + 1) ** 2 / float(gaussian_sd ** 2)))
for si in range(X.shape[0]):
    for ri in range(X.shape[1]):
        X[si, ri, :] = np.convolve(X[si, ri, :], gaussian, 'same')
    for ci in range(X.shape[2]):
        X[si, :, ci] = np.convolve(X[si, :, ci], gaussian, 'same')

perms = 100
mesh = np.stack([X, Y]).reshape(-1, 40, 40)
mesh = np.array([np.random.permutation(mesh) for i in range(perms)])
mesh = mesh.reshape(perms, 2, 10, 40, 40)
Xm = mesh[:, 0]
Ym = mesh[:, 1]
mesh = None

perms_adj = sparse.coo_array((perms, perms), dtype=int)
# perms_adj = perms
conn_adj = mne.stats.combine_adjacency(40, 40)
# perms_adj = perms
adj = mne.stats.combine_adjacency(perms_adj, conn_adj)


tstat = stats.ttest_ind(Xm, Ym, axis=1).statistic.flatten()

clus, sums = find_clusters(tstat, threshold=stats.t.ppf(0.95, 9), adjacency=adj, tail=-1)
clus, sums = list(zip(*[(cl, sum) for cl, sum in zip(clus, sums) if len(cl) >= 2]))

perm_inds = np.arange(perms).repeat(1600).flatten()

clus_inds = []
for cl in clus:
    temp = perm_inds[cl]
    clus_inds.append(np.all(temp == temp[0]))
    print(temp)
np.all(clus_inds)

[0 0 0 ... 0 0 0]
[1 1 1 ... 1 1 1]
[2 2 2 ... 2 2 2]
[3 3 3 ... 3 3 3]
[4 4 4 ... 4 4 4]
[5 5 5 ... 5 5 5]
[6 6 6 ... 6 6 6]
[7 7 7 ... 7 7 7]
[8 8 8 ... 8 8 8]
[9 9 9]
[9 9 9 ... 9 9 9]
[10 10 10 ... 10 10 10]
[11 11 11 ... 11 11 11]
[12 12 12 ... 12 12 12]
[13 13 13 ... 13 13 13]
[14 14 14 ... 14 14 14]
[15 15 15 ... 15 15 15]
[15 15]
[16 16 16 ... 16 16 16]
[17 17 17 ... 17 17 17]
[18 18 18 ... 18 18 18]
[19 19 19 ... 19 19 19]
[20 20 20 ... 20 20 20]
[21 21 21 ... 21 21 21]
[22 22 22 ... 22 22 22]
[23 23 23 ... 23 23 23]
[24 24 24 ... 24 24 24]
[24 24]
[25 25 25 ... 25 25 25]
[26 26 26 ... 26 26 26]
[27 27 27 ... 27 27 27]
[28 28 28 ... 28 28 28]
[28 28]
[29 29 29 ... 29 29 29]
[30 30 30 ... 30 30 30]
[31 31 31 ... 31 31 31]
[32 32 32 ... 32 32 32]
[33 33 33 ... 33 33 33]
[34 34 34 ... 34 34 34]
[35 35 35 ... 35 35 35]
[35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35 35]
[36 36 36 ... 36 36 36]
[37 37 37 ... 37 37 37]
[38 38 38 ... 38 38 38]
[39 39 39 39 39 39 39 39 39 39 39 39

True